This notebook shows the behavior of the implemented models using a largely reduced dataset (taking only 10 time step for demand data).

In [21]:
# Load modules
include("../src/simulation.jl")
include("../src/heuristics.jl")
include("../src/utils.jl")
include("../src/topology_optimization.jl")
include("../src/optimization.jl")

using .DCOPF
using .Utils
using .OptNet

In [9]:
# Load data
data_dir = "/home/esteban_nb/dirhw/mae573-term-project/data"
output_dir = "/home/esteban_nb/dirhw/mae573-term-project/output"

data = Utils.load_data(data_dir, shrink=true)

Loading data from /home/esteban_nb/dirhw/mae573-term-project/data...

----- Dataset Information -----

Dataset: Bus
  Rows: 120
  Columns: 5
  Column Names: ["bus_id", "zone_id", "pd", "qd", "idx"]

Dataset: Branch
  Rows: 738
  Columns: 27
  Column Names: ["start_bus", "end_bus", "branch_id", "r", "x", "b", "ratea", "rateb", "ratec", "ratio", "angle", "status", "angmin", "angmax", "pf", "qf", "pt", "qt", "mu_sf", "mu_st", "mu_angmin", "mu_angmax", "branch_device_type", "interconnect", "sus", "start_idx", "end_idx"]

Dataset: Plant
  Rows: 548
  Columns: 33
  Column Names: ["plant_id", "bus_id", "pg", "qg", "qmax", "qmin", "vg", "mbase", "status", "pmax", "pmin", "pc1", "pc2", "qc1min", "qc1max", "qc2min", "qc2max", "ramp_agc", "ramp_10", "ramp_30", "ramp_q", "apf", "mu_pmax", "mu_pmin", "mu_qmax", "mu_qmin", "type", "interconnect", "genfuelcost", "geniob", "genioc", "geniod", "bus_idx"]

Dataset: Load
  Rows: 10
  Columns: 118
  Column Names: ["Date", "Time", "1079", "2002", "2011", "

Dict{String, DataFrame} with 5 entries:
  "branch"  => 738×27 DataFrame…
  "load"    => 10×118 DataFrame…
  "bus"     => 120×5 DataFrame…
  "plant"   => 548×33 DataFrame…
  "gencost" => 548×9 DataFrame…

In [10]:
println("Executing simple DCOPF...\n")
execution_time = @elapsed results = DCOPF.perform_optimization(data)
println("Execution time: $execution_time seconds")

Executing simple DCOPF...

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2575217
Academic license 2575217 - for non-commercial use only - registered to en___@princeton.edu
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2575217 - for non-commercial use only - registered to en___@princeton.edu
Optimize a model with 23934 rows, 19540 columns and 73078 nonzeros
Model fingerprint: 0xe28f8cec
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e+00, 4e+01]
  Bounds range     [4e-01, 1e+03]
  RHS range        [3e-01, 4e+03]
Presolve removed 19187 rows and 14359 columns
Presolve time: 0.07s
Presolved: 4747 rows, 8898 columns, 24131 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...



In [11]:
function load_clean_data()
    clean_data_dir = joinpath(data_dir, "newdata")
    
    bus = CSV.read(joinpath(clean_data_dir, "bus.csv"), DataFrame)
    branch = CSV.read(joinpath(clean_data_dir, "branch.csv"), DataFrame)
    plant = CSV.read(joinpath(clean_data_dir, "plant.csv"), DataFrame)
    load = CSV.read(joinpath(clean_data_dir, "load.csv"), DataFrame)
    gencost = CSV.read(joinpath(clean_data_dir, "gencost.csv"), DataFrame)

    return Dict(
        "bus" => bus,
        "branch" => branch,
        "plant" => plant,
        "load" => load,
        "gencost" => gencost
    )
end

load_clean_data (generic function with 1 method)

In [ ]:
include("../src/topology_optimization.jl")
using .OptNet

clean_data = load_clean_data()
num_scenarios = 10 

scenarios = [Dict("line_failure_prob" => 0.05) for _ in 1:num_scenarios]
probabilities = fill(1/num_scenarios, num_scenarios)

# max modifs should be at least over line_failure_prob * num_lines (470)
results = OptNet.optimize_network(clean_data, scenarios, probabilities, 25)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2575217
Academic license 2575217 - for non-commercial use only - registered to en___@princeton.edu
Hey! 1Hey! 2

In [6]:
# Criticality assessment
# clean_data = load_clean_data()
# results = assess_criticality(clean_data)

# sorted_results = sort(results_df, :cost_after_removal)

# output_file = joinpath(output_dir, "sorted_results.csv")
# CSV.write(output_file, sorted_results)[1]

# plot(sorted_results.line, sorted_results.cost_after_removal, 
#      xlabel="Branch Line", ylabel="Cost After Removal", 
#      title="Criticality of Nodes", 
#      label="Cost After Removal", 
#      marker=:circle, 
#      legend=:topright)
# fig_path = joinpath(output_dir, "fig/criticality_plot.png")
# savefig(fig_path)

LoadError: UndefVarError: `DCOPF` not defined in `Main`
Suggestion: check for spelling errors or missing imports.